# XLM-R + Adversarial Training & Sentiment Analysis
### Hindi-English

---

## **Installations and Preparation**
Code blocks below for downloading files, installing dependencies and setting up notebook.

In [ ]:
# Installations:
!pip install wget
!pip3 install transformers

In [ ]:
# If on Google Collab:
from google.colab import drive
drive.mount('/content/drive')

# These lines for Google Collab
%cd drive/My\ Drive/NLP_finalproj/NLP_finalproj
!ls

In [ ]:
# These lines for downloading the repo+datasets from github if not already
import os
import wget

# print('Downloading dataset...')
#
# # The URL for the dataset zip file.
# url = 'https://github.com/rsgoss/NLP_finalproj/archive/main.zip'
#
# # Download the file (if we haven't already)
# if not os.path.exists('./main.zip'):
#     wget.download(url, './main.zip')
#
# # Unzip the dataset (if we haven't already)
# if not os.path.exists('./main/'):
#     !unzip main.zip

In [ ]:
# import our packages...
import tensorflow as tf
import torch

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

## Parsing Data

In [ ]:
import pandas as pd

# Load the datasets into a pandas dataframe.
df_train = pd.read_csv("data/hindi-english/train_14k_split.csv" )
df_val = pd.read_csv("data/hindi-english/val_3k_split.csv" )

# Report the number of sentences.
print('Number of training sentences in training set: {:,}\n'.format(df_train.shape[0]))
print('Number of training sentences in val set: {:,}\n'.format(df_val.shape[0]))

# Get the lists of sentences and their labels for train and val datasets
sentences_train = df_train.sentence.values
labels_train = df_train.label.values
sentences_val = df_val.sentence.values
labels_val = df_val.label.values

print('Train dataset: \n', df_train.head())
print('Val dataset: \n',df_val.head())

## Tokenizing Data
1. Tokenize the sentence.
2. Prepend the `[CLS]` token to the start.
3. Append the `[SEP]` token to the end.
4. Map tokens to their IDs.
5. Pad or truncate the sentence to `max_length`
6. Create attention masks for [PAD] tokens.

In [ ]:
from transformers import XLMRobertaTokenizer

# Load the BERT tokenizer.
print('Loading XLMRobertaTokenizer ...')
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base', do_lower_case=True)
tokenized_train = tokenizer.tokenize(sentences_train[0])
tokenized_ids_train = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences_train[0]))
tokenized_val = tokenizer.tokenize(sentences_val[0])
tokenized_ids_val = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences_val[0]))

# Print the original train sentence; tokenized and IDs mapped.
print('Original Train Sentence: ', sentences_train[0])
print('Tokenized Train Sentence: ', tokenized_train)
print('Token Train IDs: ', tokenized_ids_train)

# Print the original val sentence; tokenized and IDs mapped.
print('Original Val Sentence: ', sentences_val[0])
print('Tokenized Train Sentence: ', tokenized_val)
print('Token Train IDs: ', tokenized_ids_val)

In [ ]:
import numpy as np

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_train = []
attention_masks_train = []
input_ids_val = []
attention_masks_val = []

# Loop through sentences for Train dataset
for sent in sentences_train:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 160,
                        padding = 'max_length',
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )

    # Add the encoded sentence to the list.
    input_ids_train.append(encoded_dict['input_ids'])

    # Add its attention mask (simply differentiates padding from non-padding).
    attention_masks_train.append(encoded_dict['attention_mask'])

# Loop through sentences for Val dataset
for sent in sentences_val:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 160,
                        padding = 'max_length',
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )

    input_ids_val.append(encoded_dict['input_ids'])
    attention_masks_val.append(encoded_dict['attention_mask'])
    
# Convert the lists into tensors.
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)
labels_train = torch.tensor(labels_train)

input_ids_val = torch.cat(input_ids_val, dim=0)
attention_masks_val = torch.cat(attention_masks_val, dim=0)
labels_val = torch.tensor(labels_val)

print('Original train sentence \n: ', sentences_train[0])
print('Train Token IDs \n:', input_ids_train)
print('Train labels: \n', labels_train)
print('Train sentence shape:', np.shape(sentences_train))
print('Train token shape:', np.shape(input_ids_train))
print('Train token mask shape:', np.shape(attention_masks_train))
print('Train label shape:', np.shape(labels_train))
print('\n')
print('Original val sentence \n: ', sentences_val[0])
print('Val Token IDs \n:', input_ids_val[0])
print('Val labels: \n', labels_val)
print('Train sentence shape:', np.shape(sentences_val))
print('Train token shape:', np.shape(input_ids_val))
print('Train token mask shape:', np.shape(attention_masks_val))
print('Train label shape:', np.shape(labels_val))


## Prepare the Training and Validation Sets
Create model to be fine-tuned. Edit settings for training and validation.

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
val_dataset = TensorDataset(input_ids_val, attention_masks_val, labels_val)

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import XLMRobertaForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = XLMRobertaForSequenceClassification.from_pretrained(
    "xlm-roberta-base", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The XLMRoberta model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

## Optimizer & Learning Rate Scheduler
Now that we have our model loaded we need to grab the training hyperparameters from within the stored model. For the purposes of fine-tuning, the authors recommend choosing from the following values (from Appendix A.3 of the BERT paper):
- Batch size: 16, 32
- Learning rate (Adam): 5e-5, 3e-5, 2e-5
- Number of epochs: 2, 3, 4

We chose:
- Batch size: 32 (set when creating our DataLoaders)
- Learning rate: 3e-5
- Epochs: 4 (we'll see that this is probably too many...)
- The epsilon parameter eps = 1e-8 is "a very small number to prevent any division by zero in the implementation".

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 5

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

## **Train Model**
Training should be done on a GPU

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

## Gradient Sanity Check

In [ ]:
from transformers.modeling_utils import create_position_ids_from_input_ids
import matplotlib.pyplot as plt
import numpy as np

**experiments**

In [ ]:
 import torch.nn.functional as F

In [ ]:
x = input_ids[0].unsqueeze(0).to(device)
y=labels[0].unsqueeze(0).to(device)
ind=2

In [ ]:
tokenizer.decode(x[0])

In [ ]:
emb_matrix = model.roberta.embeddings.word_embeddings.weight
normed_emb_matrix=F.normalize(emb_matrix, p=2, dim=1)

In [ ]:
def neighboors(embedd,numb):
  normed_emb_word=F.normalize(embedd, p=2, dim=0)
  cosine_similarity = torch.matmul(normed_emb_word, torch.transpose(normed_emb_matrix,0,1))
  calc, closest_words = torch.topk(cosine_similarity,numb,dim=0)
  print(tokenizer.decode(closest_words))
  return closest_words

In [ ]:
def replace(x,ind,word):
  x[0][ind]=word
  return x

In [ ]:
model(replace(x,ind,int(closest_words[0])),labels=y)

In [ ]:
model(replace(x,ind,73),labels=y)

In [ ]:
tokenizer.decode(torch.Tensor([73]))

In [ ]:
def descent(x,y,ind):
  emb=predict1(x)
  att=PGDAttack(predict2, loss_fn=None, eps=0.3, nb_iter=100,
            eps_iter=0.005, rand_init=True, clip_min=-1., clip_max=1.,
            ord=np.inf, l1_sparsity=None, targeted=False)
  rval, norm_memory, loss_memory =att.perturb(x,emb,ind,y)
  return rval

In [ ]:
embedd=descent(x,y,ind)[0][ind]
closest_words0=neighboors(embedd,10)

In [ ]:
embedd2=list(model.roberta.embeddings.children())[:1][0](x)[0][ind] #batch:0, word:ind
closest_words=neighboors(embedd2,10)

In [ ]:
closest_words

In [ ]:
cand=torch.empty((10,len(embedd2)))
for t in range(10):
  cand[t]=list(model.roberta.embeddings.children())[:1][0](closest_words[t].unsqueeze(0))
cand=cand.to(device)

In [ ]:
cand

In [ ]:
emb=predict1(x)

In [ ]:
a, b, c = emb.size()
d=cand.size()[0]
embdelt = torch.zeros(d,a,b,c).to(device)
for jj in range(d):
  embdelt[jj][0][ind]=cand[jj]
embdelt=embdelt.permute(1,2,3,0)

In [ ]:
embdelt

In [ ]:
att=PGDAttack(predict2, loss_fn=None, eps=0.3, nb_iter=100,
            eps_iter=0.0001, rand_init=True, clip_min=-1., clip_max=1.,
            ord=np.inf, l1_sparsity=None, targeted=False)

In [ ]:
rval, norm_memory, loss_memory =att.perturb_cand(x,emb,cand,embdelt,ind,y)

In [ ]:
torch.randn(4)

**gradient descent on one input WITHOUT advertorch**